<a href="https://colab.research.google.com/github/SanjayJanardhan-89/ApacheSparkHandsOn/blob/main/Certification/Chapter_5_Advanced_Operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Pyspark


In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark
from pyspark.sql import SparkSession

spark= SparkSession \
       .builder \
       .appName("OurSparkApp") \
       .getOrCreate()

spark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [4,420 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [55.7 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updat

In [2]:
salary_data = [("John", "Field-eng", 3500),
    ("Michael", "Field-eng", 4500),
    ("Robert", None, 4000),
    ("Maria", "Finance", 3500),
    ("John", "Sales", 3000),
    ("Kelly", "Finance", 3500),
    ("Kate", "Finance", 3000),
    ("Martin", None, 3500),
    ("Kiran", "Sales", 2200),
    ("Michael", "Field-eng", 4500)
  ]
columns= ["Employee", "Department", "Salary"]
salary_data = spark.createDataFrame(data = salary_data, schema = columns)
salary_data.printSchema()
salary_data.show()

root
 |-- Employee: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Salary: long (nullable = true)

+--------+----------+------+
|Employee|Department|Salary|
+--------+----------+------+
|    John| Field-eng|  3500|
| Michael| Field-eng|  4500|
|  Robert|      NULL|  4000|
|   Maria|   Finance|  3500|
|    John|     Sales|  3000|
|   Kelly|   Finance|  3500|
|    Kate|   Finance|  3000|
|  Martin|      NULL|  3500|
|   Kiran|     Sales|  2200|
| Michael| Field-eng|  4500|
+--------+----------+------+



In [3]:
salary_data.groupBy('Department')

GroupedData[grouping expressions: [Department], value: [Employee: string, Department: string ... 1 more field], type: GroupBy]

In [6]:
salary_data.groupBy().avg().show()

+-----------+
|avg(Salary)|
+-----------+
|     3520.0|
+-----------+



In [7]:
salary_data.groupBy('Department').avg().show()

+----------+------------------+
|Department|       avg(Salary)|
+----------+------------------+
|     Sales|            2600.0|
| Field-eng| 4166.666666666667|
|      NULL|            3750.0|
|   Finance|3333.3333333333335|
+----------+------------------+



In [11]:
from pyspark.sql.functions import lit
salary_data = salary_data.withColumn('AnotherCol', lit(100))

In [12]:
salary_data.groupBy('Department').avg().show()

+----------+------------------+---------------+
|Department|       avg(Salary)|avg(AnotherCol)|
+----------+------------------+---------------+
|     Sales|            2600.0|          100.0|
| Field-eng| 4166.666666666667|          100.0|
|      NULL|            3750.0|          100.0|
|   Finance|3333.3333333333335|          100.0|
+----------+------------------+---------------+



In [15]:
salary_data = salary_data.drop('AnotherCol')

In [16]:
salary_data.show()

+--------+----------+------+
|Employee|Department|Salary|
+--------+----------+------+
|    John| Field-eng|  3500|
| Michael| Field-eng|  4500|
|  Robert|      NULL|  4000|
|   Maria|   Finance|  3500|
|    John|     Sales|  3000|
|   Kelly|   Finance|  3500|
|    Kate|   Finance|  3000|
|  Martin|      NULL|  3500|
|   Kiran|     Sales|  2200|
| Michael| Field-eng|  4500|
+--------+----------+------+



In [25]:
from pyspark.sql.functions import *

(salary_data
  .groupBy("Department").sum("salary")
  .withColumn("sum(salary)", round(col("sum(salary)"),4))
  .withColumnRenamed("sum(salary)", "sum_salary")
  .orderBy(desc("sum_salary"))
  .show()
)

+----------+----------+
|Department|sum_salary|
+----------+----------+
| Field-eng|     12500|
|   Finance|     10000|
|      NULL|      7500|
|     Sales|      5200|
+----------+----------+

